In [1]:
# Import the bitstream
from pynq import Overlay
ol_adds = Overlay("overlay/harness_axis.bit")

In [2]:
from time import time
num_data = 1000

In [3]:
# Allocate input and output buffer
import numpy as np
from pynq import allocate

in_buffer = allocate(shape=(num_data,), dtype=np.uint32, cacheable=1)
out_buffer = allocate(shape=(num_data,), dtype=np.uint32, cacheable=1)

In [4]:
# Assign random values to input buffer
for i in range(num_data):
    in_buffer[i] = (np.random.randint(0,15)<<4)+np.random.randint(0,15)

In [5]:
# Print the values of the input buffer
for i in range(10):
    print(hex(in_buffer[i]))

0xe
0x4c
0xad
0x16
0x55
0xa3
0x27
0xea
0xed
0x28


In [6]:
# Print the values of the output buffer. All the values will be zero.
for i in range(10):
    print('0x' + format(out_buffer[i], '02x'))

0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00
0x00


In [7]:
# DMA driver
import pynq.lib.dma
dma_ip = ol_adds.dma

In [8]:
# Function to call our axi_adder IP
def adds_data():
    dma_ip.sendchannel.transfer(in_buffer)
    dma_ip.recvchannel.transfer(out_buffer)
    dma_ip.sendchannel.wait()
    dma_ip.recvchannel.wait()

In [9]:
# Clocks
# Note we are using fclk1_mhz for our axi_adder and fclk0_mhz for all other IP blocks.
from pynq import Clocks

print(f'CPU:   {Clocks.cpu_mhz:.6f}MHz')
print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')
print(f'FCLK1: {Clocks.fclk1_mhz:.6f}MHz')

CPU:   650.000000MHz
FCLK0: 50.000000MHz
FCLK1: 200.000000MHz


In [10]:
# Run
t1 = time()
adds_data()
t2 = time()
t_adds = t2-t1
print(f'Time taken for AXI adder IP running at {Clocks.fclk1_mhz:.6f}MHz', t_adds)

Time taken for AXI adder IP running at 200.000000MHz 0.0020689964294433594


In [11]:
# Print the output buffer 
for i in range(10):
    print('0x' + format(out_buffer[i], '02x'))

0x0e
0x10
0x17
0x07
0x0a
0x0d
0x09
0x18
0x1b
0x0a


In [12]:
# Increase the axi adder clock frequency
Clocks.fclk1_mhz = 500

In [13]:
# Run again
t1 = time()
adds_data()
t2 = time()
t_adds = t2-t1
print(f'Time taken for AXI adder IP running at {Clocks.fclk1_mhz:.6f}MHz', t_adds)

Time taken for AXI adder IP running at 500.000000MHz 0.0027146339416503906


In [14]:
# Print the output buffer 
for i in range(10):
    print('0x' + format(out_buffer[i], '02x'))

0x0e
0x05
0x10
0x17
0x06
0x07
0x0a
0x05
0x0d
0x09
